In [1]:
import ptmpsi
from ptmpsi.alphafold import prediction
from ptmpsi.protein import Protein
from ptmpsi.nwchem import get_qm_data
from ptmpsi.polymers.tools import gen_pdb

# Structure Prediction with AlphaFold

## Pass a FASTA sequence to the `prediction` function

In [2]:
prediction("GGGGGGGGGGG", multimer=False, machine='frontier', account='bip258')

	 Info: Frontier will use experimental container for alphafold 2.3.2 from /sw/frontier/preview/modulefiles/

	 Info: Make sure to copy alphafold.sbatch, run_singularity.py, your
	       fasta files and all generated temp*.fasta files to a submission
	       directory and execute `sbatch alphafold.sbatch`


Three files will be generated in the current working directory: `alphafold.sbatch`, `run_singularity.py` and `temp1.fasta`.
Paths to the required databases will default to their location in the Tahoma cluster.

The `prediction` function also accepts a list of sequences for the sequential prediction for each one of the sequences.

If **multimer** is instead desired, one must pass `multimer=True` to the `prediction` and a path to 
a file containing two or more sequences with the following format:

```
>sequence_1
GGGGGGGG

>sequence_2
AAAAAAAA
```

# Loading a predicted 3D structure

The `Protein` class handles all protein/polymer operations. We will use this class to load or build the 3D structure 
of a polymer. 

## Read a structure from a local PDB file
Create a `Protein` object using the `filename` option to pass the path to a local PDB file

In [13]:
local_pdb = Protein(filename="G9BY57.pdb")

	 Reading local PDB file


## Fetch a structure from the Protein Data Bank
Only works when internet access is available. Create a `Protein` object using the `pdbid` option to pass a Protein Data Bank ID code.

In [14]:
fetched_pdb = Protein(pdbid="6tht")

		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'A' with occupancy   0.63
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy

## Fetch a structure from the AlphaFold Database
Only works when internet access is available. Create a `Protein` object using the `uniprotid` option to pass a Uniprot ID code.

In [15]:
uniprot_pdb = Protein(uniprotid="G9BY57")

KeyboardInterrupt: 

## Build a chain from scratch
Create an empty `Protein` object and use the `append` and `prepend` functions to add residues to a given polymer *chain*. The `phi`/`psi` amide torsions can be customized while using the `prepend`/`append` commands, respectively. The default values will produce torsion angles compatible with the alpha-helix conformation. 

In [ ]:
chain = Protein()

# Build a trimer of epsilon-amino caproic acid
chain.prepend(chain="A", residue="EAC")
chain.append(chain="A", residue="EAC", psi=120.0)
chain.append(chain="A", residue="EAC", psi=120.0)

# Cap the trimer to have neutral ends
chain.prepend(chain="A", residue="ACE", phi=-120.0)
chain.append(chain="A", residue="NME", psi=120.0)

## Generate a Nylon-6 surface
Use the `gen_pdb` function to create a PDB file of a finite cluster approximation of a Nylon-6 surface by specifying the number of unit cells to include along the `a`, `b`, and `c` directions. Two PDB files will be generated, one where all of the strands share the same *MODEL* parameter and another where each strand is its own *MODEL*. There is no difference among the models, but some visualization software packages might get confused with many strands in one *MODEL*.

In [ ]:
gen_pdb(rangea=3, rangeb=3, rangec=3)

TypeError: gen_pdb() got an unexpected keyword argument 'rangea'

# Introducing point mutations

The `mutate` function of the `Protein` class can be used to introduce point mutations in a protein. For example, the active form of the ICCG variant of cutinase can be obtained by the following commands.

In [ ]:
cutinase = Protein(pdbid="6tht")
cutinase.mutate("A:ALA130","SER")

KeyboardInterrupt: 

The residue to be mutated can be specified with the nomenclature "\[chain\]:\[name\]\[number\]", where \[chain\] is the one-letter chain identifier, \[name\] is the three-letter aminoacid name, and \[number\] is the residue number. The \[name\] can be omitted.

# Obtaining AMBER99 parameters for a residue

The `get_qm_data` function will produce a series of `NWChem` input files that will generate `RESP` charges and bonded force constants according to the `AMBER99` forcefield recipe. The current output is obtained in `STDOUT` and the force constants are given in a format that can be copied directly into a `GROMACS` forcefield definition.

By default, `get_qm_data` will generate two conformations for the new residue capped by an `ACE` and `NME` groups. The `RESP` fitting for both conformers will be performed simultaneously and only one set of charges will be computed. Bond and angle force constants parameters will be generated **separately** and the user must average these values in order to obtain a single constant for each bond or angle.

When needed, torsion potentials are fitted for both conformations at the same time.

In [2]:
eac = Protein()
eac.prepend(chain="A", residue="EAC")
get_qm_data(eac, machine='frontier', account='bip258')


Some NWChem input options can be changed via keyword arguments. These options, along with their default values, are:

```
mult    = kwargs.get("mult",1)
charge  = kwargs.get("charge",0)
memory  = kwargs.get("memory",2000)
aobasis = kwargs.get("aobasis","def2-tzvp")
tdbasis = kwargs.get("tdbasis","def2-svp")
cdbasis = kwargs.get("cdbasis","def2-universal-jfit")
xcfun   = kwargs.get("xcfun","r2scan")
grid    = kwargs.get("grid","lebedev 120 14")
tdgrid  = kwargs.get("tdgrid","lebedev 100 14")
nscf    = kwargs.get("nscf",100)
nopt    = kwargs.get("nopt",60)
disp    = kwargs.get("disp","disp vdw 4")
delta   = kwargs.get("delta",0.0189)
lshift  = kwargs.get("lshift",0.1)
```

# Post-translational modifications

Post-translational modifications can be introduced by using the `modify` function of the `Protein` class. The following is a list of available CYS post-translational modifications
 - nitrosylation
 - sulfenylation
 - sulfynilation
 - sulfonylation
 - sulfhydration
 - glutathionylation
 - cysteinylation
 - methylation
 - carbamoylation
 - cyanylation

The following post-translational modifications are also available for a subset of aminoacids
 - acetylation
     - LYS, LYN
 - methylation:
     - GLU, GLH
     - LYS, LYN
     - ARG
     - HIS, HIP, HID, HIE
 - dimethylation:
     - LYS, LYN, ARG
 - trimethylation:
     - LYS, LYN
 - symmetric dimethylation:
     - ARG
 - asymmetric dimethylation:
     - ARG        
 - phosphorylation:
     - SER
     - THR
     - TYR
     - ARG
     - HIS, HIP, HID, HIE
     - LYS, LYN
     - ASP, ASH
     - CYS, CYM

In [ ]:
snc = Protein()
snc.prepend(chain="A", residue="CYS")
snc.modify("A:CYS1", "nitrosylation")

nitrosylation SNC

	 Found rotamer with no clashes!


# Docking

PTM-Psi relies on existing installations of AutoDock Vina and the AutoDock Tools packages. PTM-Psi will not attempt to fetch these packages. 

In [2]:
cutinase = Protein(pdbid="6tht")
cutinase.mutate("A:ALA130","SER")
cutinase.write_pdb("cutinase.pdb")

nylon6 = Protein()
nylon6.prepend("A", "EAC")
nylon6.prepend("A", "ACE")
nylon6.append("A", "EAC")
nylon6.append("A", "NME")
nylon6.write_xyz("nylon6.xyz")
cutinase.dock(ligand="nylon6.xyz", receptor="cutinase.pdb", boxcenter="A:SER130", boxsize=20.0, exhaustiveness=64)

		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'A' with occupancy   0.63
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy

NameError: name 'MyDockingError' is not defined

# NWChem QM/MM input generation

# Generate post-translational modification combinations

- Load the GAP2/CP12/PRK complex

In [5]:
# protein = Protein(pdbid="6gve")
protein = Protein(filename="6gve.pdb")

	 Reading local PDB file


- Find all cysteine residues

In [6]:
protein.findresidue("CYS")

D:CYS19
D:CYS67
D:CYS77
D:CYS154
D:CYS158
A:CYS19
A:CYS67
A:CYS77
A:CYS154
A:CYS158
E:CYS19
E:CYS67
E:CYS77
E:CYS154
E:CYS158
F:CYS19
F:CYS67
F:CYS77
F:CYS154
F:CYS158
I:CYS19
I:CYS67
I:CYS77
I:CYS154
I:CYS158
K:CYS19
K:CYS67
K:CYS77
K:CYS154
K:CYS158
N:CYS19
N:CYS67
N:CYS77
N:CYS154
N:CYS158
O:CYS19
O:CYS67
O:CYS77
O:CYS154
O:CYS158


- Generate all posible single combinations (ntuple=1) for nitrosylation and sulfhydration

In [7]:
combinations1 = protein.get_ptm_combinations([("CYS", "nitrosylation"),("CYS","sulfhydration")], ntuple=1)

	Number of 1-tuple combinations: 80
Total number of combinations: 80


- List all generated combinations

In [8]:
print("Residue         PTM")
for ptm in combinations1[0]:
    print(f"{ptm[0][0]:10s}   {ptm[0][1]:20s}")

Residue         PTM
D:CYS19      nitrosylation       
D:CYS67      nitrosylation       
D:CYS77      nitrosylation       
D:CYS154     nitrosylation       
D:CYS158     nitrosylation       
A:CYS19      nitrosylation       
A:CYS67      nitrosylation       
A:CYS77      nitrosylation       
A:CYS154     nitrosylation       
A:CYS158     nitrosylation       
E:CYS19      nitrosylation       
E:CYS67      nitrosylation       
E:CYS77      nitrosylation       
E:CYS154     nitrosylation       
E:CYS158     nitrosylation       
F:CYS19      nitrosylation       
F:CYS67      nitrosylation       
F:CYS77      nitrosylation       
F:CYS154     nitrosylation       
F:CYS158     nitrosylation       
I:CYS19      nitrosylation       
I:CYS67      nitrosylation       
I:CYS77      nitrosylation       
I:CYS154     nitrosylation       
I:CYS158     nitrosylation       
K:CYS19      nitrosylation       
K:CYS67      nitrosylation       
K:CYS77      nitrosylation       
K:CYS154     nitrosylation  

- Generate all possible single and double combinations excluding modification of all CYS19 and CYS77 of GAP2

In [9]:
exclude = [ f"{chain}:{res}" for chain in ["A", "D", "E", "F", "I", "K", "N", "O"] for res in ["CYS19", "CYS77"] ]
combinations2 = protein.get_ptm_combinations([("CYS", "nitrosylation"),("CYS","sulfhydration")], exclude=exclude, ntuple=2)

	Number of 1-tuple combinations: 48
	Number of 2-tuple combinations: 1104
Total number of combinations: 1152


- Find all cysteine residues involved in disulfide bonds (identifed as CYX)

In [10]:
protein.findresidue("CYX")

P:CYX19
P:CYX29
P:CYX61
P:CYX70
J:CYX13
J:CYX35
J:CYX224
J:CYX230
B:CYX13
B:CYX35
B:CYX224
B:CYX230
G:CYX13
G:CYX35
G:CYX224
G:CYX230
L:CYX13
L:CYX35
L:CYX224
L:CYX230
C:CYX19
C:CYX29
C:CYX61
C:CYX70
H:CYX19
H:CYX29
H:CYX61
H:CYX70
M:CYX19
M:CYX29
M:CYX61
M:CYX70


- Generate a combination list that reduces both disulfide bonds of CP12 chain M

In [11]:
combinations3 = protein.get_ptm_combinations([("M:CYX", "reduction")], ntuple=4)

	Number of 1-tuple combinations: 4
	Number of 2-tuple combinations: 6
	Number of 3-tuple combinations: 4
	Number of 4-tuple combinations: 1
Total number of combinations: 15


In [12]:
print("1-tuple")
print("     Residue         PTM")
for i,ptm in enumerate(combinations3[0]):
    for residue in ptm:
        print(f"{i+1:3d}  {residue[0]:10s}   {residue[1]:20s}")

print()
print("2-tuple")
print("     Residue1       PTM1     Residue2       PTM2")
for i,ptm in enumerate(combinations3[1]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()

print()
print("3-tuple")
print("     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3")
for i,ptm in enumerate(combinations3[2]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()

print()
print("4-tuple")
print("     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3      Residue4      PTM4")
for i,ptm in enumerate(combinations3[3]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()


1-tuple
     Residue         PTM
  1  M:CYX19      reduction           
  2  M:CYX29      reduction           
  3  M:CYX61      reduction           
  4  M:CYX70      reduction           

2-tuple
     Residue1       PTM1     Residue2       PTM2
  1   M:CYX19    reduction    M:CYX29    reduction    
  2   M:CYX19    reduction    M:CYX61    reduction    
  3   M:CYX19    reduction    M:CYX70    reduction    
  4   M:CYX29    reduction    M:CYX61    reduction    
  5   M:CYX29    reduction    M:CYX70    reduction    
  6   M:CYX61    reduction    M:CYX70    reduction    

3-tuple
     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3
  1   M:CYX19    reduction    M:CYX29    reduction    M:CYX61    reduction    
  2   M:CYX19    reduction    M:CYX29    reduction    M:CYX70    reduction    
  3   M:CYX19    reduction    M:CYX61    reduction    M:CYX70    reduction    
  4   M:CYX29    reduction    M:CYX61    reduction    M:CYX70    reduction    

4-tuple
     Residue1    

Note that some double (2-tuple) combinations and the quadruple (4-tuple) combination are the only ones that make sense for this case as CYX19 is bonded to CYX29 and CYX61 is bonded to CYX70.

- Generate the run files for the quadruple combination

The following command will generate a folder with a unique identifier (UID) with all the necessary files to submit a full set of GROMACS MD simulations including energy minimization, equilibration, and a 100 ns production run. Note that there is a modified **specbond.dat** file that will prevent GROMACS from forming a disulfide bond between spatially close **CYS** residues, but it will still form disulfide bonds between two nearby **CYX** residues.

In [13]:
uid = protein.gen_ptm_files([combinations3[3]], lenmd=1, lennvt=100, lennpt=100 machine="frontier", account='bip258', **ptmpsi.slurm.frontier.partitions["batch"].options["gromacs"])

	 Reading local PDB file
reduction CYS

	 Found rotamer with no clashes!
reduction CYS

	 Current rotamer has 4 possible clashes
	 Current rotamer has 4 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 1 possible clashes
	 Current rotamer has 1 possible clashes

	 Found rotamer with no clashes!
reduction CYS

	 Found rotamer with no clashes!
reduction CYS

	 Current rotamer has 2 possible clashes
	 Current rotamer has 1 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible cla